<a href="https://colab.research.google.com/github/claredavies/ArtificialIntelligience/blob/main/ComputationalIntelligienceAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy
import pandas as pd
import random

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""


'\nThe y=target is to maximize this equation ASAP:\n    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6\n    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)\n    What are the best values for the 6 weights w1 to w6?\n    We are going to use the genetic algorithm for the best possible values after a number of generations.\n'

In [2]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

# Functions

In [3]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [4]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

# Genetic Algorithm

In [5]:
# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""

sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print(new_population)

[[-1.55251828 -1.3979982  -1.20213281 -1.8012255   2.55766628  3.71146372]
 [-1.64227287 -3.38886576 -0.88735168 -1.34987763 -3.89244613 -1.83744973]
 [-2.9983706   3.22193208  2.57973894 -3.80212099  1.12189061  2.68381355]
 [-3.55145953 -3.03537685  2.41278174  0.69948298  0.25074664 -3.02056782]
 [ 3.60173967 -0.14695323  1.55157534 -3.34724826 -1.27263467  1.92846208]
 [-1.16020846 -2.6953081  -0.61584509  1.60358996  1.6721659   2.45313803]
 [ 0.28967207  2.66360769 -1.02542538 -2.45469532  1.90569564 -2.86702684]
 [ 0.95681415 -2.12077359 -0.80843791 -0.13272721  2.50684699  2.62444128]]


In [6]:
def evolutionary_algorithm(num_generations):
  for generation in range(num_generations):
      print("Generation : ", generation)
      # Measing the fitness of each chromosome in the population.
      fitness = cal_pop_fitness(equation_inputs, new_population)

      # Selecting the best parents in the population for mating.
      parents = select_mating_pool(new_population, fitness, 
                                        num_parents_mating)

      # Generating next generation using crossover.
      offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))

      # Adding some variations to the offsrping using mutation.
      offspring_mutation = mutation(offspring_crossover)

      # Creating the new population based on the parents and offspring.
      new_population[0:parents.shape[0], :] = parents
      new_population[parents.shape[0]:, :] = offspring_mutation

      # The best result in the current iteration.
      print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))

  # Getting the best solution after iterating finishing all generations.
  #At first, the fitness is calculated for each solution in the final generation.
  fitness = cal_pop_fitness(equation_inputs, new_population)
  # Then return the index of that solution corresponding to the best fitness.
  best_match_idx = numpy.where(fitness == numpy.max(fitness))

  print("Finished Iterations")
  print("Best solution : ", new_population[best_match_idx, :])
  print("Best solution fitness : ", fitness[best_match_idx])

  return new_population[best_match_idx, :]

# Inputs

Class Info

In [7]:
#Creating Series
class_series = pd.Series(['Class1', 'Class1', 'Class1', 'Class1','Class2', 'Class2', 'Class2', 'Class2'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'English', 'History', 'Physics'])
hours_series = pd.Series([3, 4, 5, 3, 2, 4, 5, 3])


#Creating Dictionary
frame = { 'Class': class_series, 'Subject': subject_series, 'Hours': hours_series }
#Creating Dataframe
class_subject_hours_dataframe = pd.DataFrame(frame)

#Printing dataframe
print(class_subject_hours_dataframe)

    Class    Subject  Hours
0  Class1      Maths      3
1  Class1    English      4
2  Class1  Chemistry      5
3  Class1    History      3
4  Class2      Maths      2
5  Class2    English      4
6  Class2    History      5
7  Class2    Physics      3


Teacher Info

In [8]:
teacher_series = pd.Series(['ProfessorA', 'ProfessorA', 'ProfessorB', 'ProfessorB','ProfessorC', 'ProfessorC'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'Physics'])

#Creating Dictionary
frame = { 'Teacher': teacher_series, 'Subject': subject_series}
#Creating Dataframe
teacher_subject_dataframe = pd.DataFrame(frame)

teacher_subject_array = teacher_subject_dataframe.to_numpy()
#Printing dataframe
print(teacher_subject_array)

[['ProfessorA' 'Maths']
 ['ProfessorA' 'English']
 ['ProfessorB' 'Chemistry']
 ['ProfessorB' 'History']
 ['ProfessorC' 'Maths']
 ['ProfessorC' 'Physics']]


Define Timetable

In [9]:
startTime = 10
durationLesson = 1
lunchTime = 12
durationLunch = 1
endTime = 15

singleTimetable = []
daysOfWeek = ['Mon','Tue','Wed', 'Thur']

for day in daysOfWeek:
  for time in range(startTime, endTime, durationLesson):
    if time != lunchTime:
      singleTimetable.append([day, time])

print(singleTimetable)

[['Mon', 10], ['Mon', 11], ['Mon', 13], ['Mon', 14], ['Tue', 10], ['Tue', 11], ['Tue', 13], ['Tue', 14], ['Wed', 10], ['Wed', 11], ['Wed', 13], ['Wed', 14], ['Thur', 10], ['Thur', 11], ['Thur', 13], ['Thur', 14]]


In [10]:
def create_timetables(class_subject_hours_dataframe, timetable):
  all_timetables = []
  blank_timetable = [None] * len(timetable)
  number_class_timetables = class_subject_hours_dataframe.Class.nunique()

  for x in range(number_class_timetables):
    all_timetables.append(blank_timetable)

  # print(all_timetables)
  all_timetables_array = numpy.array(all_timetables)
  return all_timetables_array

In [11]:
def make_array_1_hour_class(class_mask):
  list_classes_1hr = []
  for index, row in class_mask.iterrows():
    number_hours_per_class = row['Hours']
    for x in range(number_hours_per_class):
      list_classes_1hr.append(row['Subject'])
  list_finished = list_classes_1hr.copy()
  return list_finished

# Genetic Fucntions Timetable

In [12]:
def generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, timetable, populationSize):
  population = []
  class_names = class_subject_hours_dataframe.Class.unique()
  number_class_names = class_subject_hours_dataframe.Class.nunique()

  for a in range(populationSize):
    trackClass = 0
    all_timetables = create_timetables(class_subject_hours_dataframe, timetable)
    for class_name in class_names:
      mask = class_subject_hours_dataframe.loc[class_subject_hours_dataframe['Class'] == class_name]
      list_classes_1hr = make_array_1_hour_class(mask)
      list_indices_single_timetable = [None] * len(timetable)

      if(len(list_classes_1hr) > len(list_indices_single_timetable) ):
        print("Not enough hours in timetable for class hours set!!")
        return

      for i in range(len(list_indices_single_timetable)):
        list_indices_single_timetable[i] = i
    
      range_end_single_timetable = len(list_indices_single_timetable) - 1

      for j in range((len(list_classes_1hr) - 1)):
        random_num = random.randint(0, range_end_single_timetable)

        random_num_index_value = list_indices_single_timetable[random_num]
        all_timetables[trackClass][random_num_index_value] = list_classes_1hr[j]
    
        list_indices_single_timetable.pop(random_num)
        range_end_single_timetable = range_end_single_timetable - 1
    
      trackClass = trackClass + 1

    population.append(all_timetables)
  return population

In [13]:
def fitnessFunction(pop):
  arrPop = numpy.array(pop)
  num_representations = arrPop.shape[0]
  NoClashesList = [None] * num_representations
 
  print("no representations:  " + str(num_representations))
  for i in range(num_representations):
    populationSelect = pop[i]
    # print(populationSelect)
    noClashes = calClashes(populationSelect)
    NoClashesList[i] = noClashes
    # print("no of clashes:  " + str(noClashes))
  # print("clash list: " + str(NoClashesList))
  return NoClashesList

In [14]:
def calClashes(pop):
  num_timetable_slots = pop.shape[1]
  num_classes = pop.shape[0]

  clash = 0
  for i in range(num_timetable_slots):
    for j in range(1, num_classes):
      if(pop[(j-1)][i] == pop[j][i]):
        if(pop[(j-1)][i] != None):
          clash = clash + 1
          # print("there is a clash at index:  " + str(i) + "  for the subject:  " + str(pop[(j-1)][i]))
  # print("clashes found:  " + str(clash))
  return clash

In [70]:
def recombine(parent1, parent2):
  parent1Array = numpy.array(parent1)
  parent2Array = numpy.array(parent2)

  noTimetables = parent2Array.shape[0]

  # print("parent 1: " + str(parent1Array))
  # print("parent 2: " + str(parent2Array))

  # print("shape: " + str(parent2Array.shape))

  crossOverIndex = int(parent1Array.shape[1]/2)
  # print("crossoverIndex: " + str(crossOverIndex))

  for i in range(noTimetables):    
    parent1FirstHalf = parent1Array[i][0:(crossOverIndex-1)]
    parent1SecondHalf = parent1Array[i][crossOverIndex:(parent1.shape[1]-1)]
    parent2FirstHalf = parent2Array[i][0:(crossOverIndex-1)]
    parent2SecondHalf = parent2Array[i][crossOverIndex:(parent1.shape[1]-1)]

    indicesListNotMatching = []
    parent1ResultFirstHalf = parent1FirstHalf
    parent1OriginalSecondHalfCopy = parent1SecondHalf
    parent1ResultSecondHalf = parent2SecondHalf
    length2ndHalf = len(parent2SecondHalf)

    parent2ResultFirstHalf = parent2FirstHalf
    parent2ResultSecondHalf = parent1SecondHalf

    print("second half parent1:  " + str(parent1SecondHalf))
    print("second half parent2:  " + str(parent1ResultSecondHalf))

    for j in range(length2ndHalf):
      print("j is:  " + str(j))
      print("Subject lookint for is : " + str(parent1ResultSecondHalf[j]))
      print("result second half: " + str(parent1ResultSecondHalf))
      print("parent 1 original second half copy: " + str(parent1OriginalSecondHalfCopy))

      if parent1ResultSecondHalf[j] not in parent1OriginalSecondHalfCopy:
        indicesListNotMatching.append(j)
        print("subject NOT found")
        print("indices not matching:  " + str(indicesListNotMatching))
        print("not in 2nd half:  " + str(parent1ResultSecondHalf[j]))
      else:
        result = numpy.where(parent1OriginalSecondHalfCopy == parent1ResultSecondHalf[j])
        parent1OriginalSecondHalfCopy = numpy.delete(parent1OriginalSecondHalfCopy, result[0][0])
        print("subject found")
        print("original second half copy after deletion: " + str(parent1OriginalSecondHalfCopy))
    
    print("Subjects missed 2nd half:  " + str(parent1OriginalSecondHalfCopy))
    print("Indices no match 2nd half:  " + str(indicesListNotMatching))
    for j in range(len(parent1OriginalSecondHalfCopy)):
      parent1ResultSecondHalf[indicesListNotMatching[j]] = parent1OriginalSecondHalfCopy[j]

    print("Final result 2nd half for first: " + str(parent1ResultSecondHalf))

    print("FINISHED TIMETABLE")

In [42]:
def evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize):
  population = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize)
  fitness = fitnessFunction(population)

  # print("fitness:  " + str(fitness))
  max_value = max(fitness)
  max_index = fitness.index(max_value)
  fitnessWithoutMax = fitness.copy()
  fitnessWithoutMax[max_index] = 0
  second_max_value = max(fitnessWithoutMax)
  second_max_index = fitness.index(second_max_value)

  firstParent = population[max_index]
  secondParent =  population[second_max_index]

  # print("max value:  " + str(firstParent))
  # print("2nd max value:  " + str(secondParent))

  recombine(firstParent, secondParent)

In [74]:
evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, 4)

no representations:  4
second half parent1:  ['English' None 'Maths' None 'Chemistry' 'Chemistry' 'English']
second half parent2:  ['Chemistry' 'Chemistry' 'Maths' 'English' 'Maths' 'History' None]
j is:  0
Subject lookint for is : Chemistry
result second half: ['Chemistry' 'Chemistry' 'Maths' 'English' 'Maths' 'History' None]
parent 1 original second half copy: ['English' None 'Maths' None 'Chemistry' 'Chemistry' 'English']
subject found
original second half copy after deletion: ['English' None 'Maths' None 'Chemistry' 'English']
j is:  1
Subject lookint for is : Chemistry
result second half: ['Chemistry' 'Chemistry' 'Maths' 'English' 'Maths' 'History' None]
parent 1 original second half copy: ['English' None 'Maths' None 'Chemistry' 'English']
subject found
original second half copy after deletion: ['English' None 'Maths' None 'English']
j is:  2
Subject lookint for is : Maths
result second half: ['Chemistry' 'Chemistry' 'Maths' 'English' 'Maths' 'History' None]
parent 1 original sec

# Execution

In [38]:
num_generations = 5

evolutionary_algorithm(num_generations)

Generation :  0
Best result :  134.30444802581837
Generation :  1
Best result :  134.30444802581837
Generation :  2
Best result :  138.95503532987385
Generation :  3
Best result :  142.72556515422528
Generation :  4
Best result :  150.93878864210043
Finished Iterations
Best solution :  [[[ -1.62969513  -2.97072974   2.30087151   2.18980049 -10.44629408
    -3.74570684]]]
Best solution fitness :  [150.93878864]


array([[[ -1.62969513,  -2.97072974,   2.30087151,   2.18980049,
         -10.44629408,  -3.74570684]]])